In this colab you will see how to install and train the lyre model (Demucs+Wav2Vec).

First, we will have to install the packages needed to run the train

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     | 1.1MB 12.8MB/s
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2336129 sha256=284a2dfa9edadb8b18e30d052544313540095dd4777fce3b601df0b86910aec6
  Stored in directory: /tmp/pip-ephem-wheel-cache-erl5_eai/wheels/2d/32/73/e3093c9d11dc8abf79c156a4db1a1c5631428059d4f9ff2cba
Successfully built kenlm


In [ ]:
!git clone https://github.com/ynop/py-ctc-decode

Cloning into 'py-ctc-decode'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 6), reused 17 (delta 2), pack-reused 0
Unpacking objects: 100% (28/28), done.


Before installing py-ctc-decode, we must remove the version dependencies in REQURED section from setup.py. Otherwise, it will install older versions of `numpy`, `tqdm` and `psutil`

```python
REQUIRED = [
    'tqdm==4.39.0',
    'kenlm @ git+ssh://git@github.com/kpu/kenlm@{}#egg=kenlm'.format(
        KENLM_COMMIT
    ),
    'numpy==1.16.2',
    'psutil==5.6.7',
]
```

to

```python
REQUIRED = [
    'tqdm',
    'kenlm @ git+ssh://git@github.com/kpu/kenlm@master',
    'numpy',
    'psutil',
]
```

In [ ]:
!sed -i -e 's/tqdm==4\.39\.0/tqdm/g' -e 's/numpy==1\.16\.2/numpy/g' -e 's/psutil==5\.6\.7/psutil/g' py-ctc-decode/setup.py

Once removed we can install the package

In [ ]:
%cd py-ctc-decode
!pip install .
%cd -

/content/py-ctc-decode
Processing /content/py-ctc-decode
  Created wheel for ctcdecode: filename=ctcdecode-0.0.0-cp37-none-any.whl size=9275 sha256=078cd37b06ee78f6f61b78e6fac5978a95586d5c2899393e8699796754ee9c01
  Stored in directory: /root/.cache/pip/wheels/b5/6a/d2/da6d2e5294501852bcb321a46602d1a684f0aee9e9be7cd24f
Successfully built ctcdecode
/root


Now it is time to clone the most relevant code in this notebook, the **lyre** package

In [ ]:
%rm -fr aidl-lyrics-recognition
!git clone https://github.com/ttecles/aidl-lyrics-recognition
%cd aidl-lyrics-recognition

Cloning into 'aidl-lyrics-recognition'...
remote: Enumerating objects: 375, done.
remote: Counting objects: 100% (375/375), done.
remote: Compressing objects: 100% (274/274), done.
remote: Total 375 (delta 225), reused 205 (delta 93), pack-reused 0
Receiving objects: 100% (375/375), 1.70 MiB | 12.03 MiB/s, done.
Resolving deltas: 100% (225/225), done.
/root/aidl-lyrics-recognition/aidl-lyrics-recognition


In [ ]:
pip install -r requirements.txt

     |████████████████████████████████| 1.9MB 40.2MB/s 
     |████████████████████████████████| 1.8MB 37.6MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
     |████████████████████████████████| 2.5MB 45.6MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 1.9MB 43.9MB/s 
     |████████████████████████████████| 143kB 56.9MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 174kB 52.2MB/s 
     |████████████████████████████████| 194kB 54.4MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
     |████████████████████████████████| 901kB 41.2MB/s 
     |████████████████████████████████| 3.3MB 41.8MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 71kB 12.3MB/s 
  Created wheel for demucs: filename=demucs-2.0.3-cp37-none-any.whl size=44136 sha256=63aabd56e1a9c8fe5380b1ddd0162483b82cbf81559b29bf04e91e2f24c19f72
  Stored in dire

Nou we will mount the drive where the DALI dataset lives

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

Let's load all the vars needed for register information into Weights & Biases

In [ ]:
env = dict(
    WANDB_KEY="PROJECT",
    WANDB_PROJECT="demucs_wav2vec",
    WANDB_ENTITY="",
    WANDB_NAME="colab run",
    WANDB_MODE="online",)
os.environ.update(env)

In [ ]:
!python -m lyre.train -h

usage: train [-h] [--data-path DATA_PATH] [--dali-gt-file DALI_GT_FILE]
             [--blacklist-file BLACKLIST_FILE] [--lm LM] [--ncc NCC]
             [--train-split TRAIN_SPLIT] [--audio-length AUDIO_LENGTH]
             [--stride STRIDE] [--epochs EPOCHS] [--batch BATCH]
             [--optimizer {adam,sgd}] [--lr LR] [--wd WD] [--fp16 | --cpu]
             [--workers WORKERS] [--demucs DEMUCS] [--wav2vec WAV2VEC]
             [--tokenizer TOKENIZER] [--freeze-demucs] [--freeze-extractor]
             [--load-model LOAD_MODEL] [--model-folder MODEL_FOLDER]
             [--save-on-epoch SAVE_ON_EPOCH]

optional arguments:
  -h, --help            show this help message and exit

data arguments:
  data input information

  --data-path DATA_PATH
                        Path holding all the data.
  --dali-gt-file DALI_GT_FILE
                        DALI ground truth file.
  --blacklist-file BLACKLIST_FILE
                        All the identifiers listed in the file will be skipped
 

Now it is time to train the model

In [ ]:
!python -m lyre.train --data-path /content/drive/MyDrive/UPC-AIDL-Spring2021/data/subset \
    --lm /content/drive/MyDrive/UPC-AIDL-Spring2021/data/text.arpa --epoch 1 --freeze-demucs --freeze-extractor \
    --model-folder /tmp

Loading DALI dataset...
Preparing Datasets...
Train DaliDataset: 30 chunks
Validation DaliDataset: 29 chunks
Test DaliDataset: 34 chunks
Loading the LM will be faster if you build a binary file.
Reading /content/drive/MyDrive/UPC-AIDL-Spring2021/data/text.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
found 1gram
found 2gram
wandb: Currently logged in as: ttecles (use `wandb login --relogin` to force relogin)
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
2021-07-11 19:57:06.956333: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.10.33
wandb: Syncing run colab run
wandb: ⭐️ View project at https://wandb.ai/aidl-lyrics-recognition/demucs%2Bwav2vec
wandb: 🚀 View run at https://wandb.ai/aidl-lyrics-

You can send a `kill -10 PID` during training and it will save the weights into the model folder 